# Testing

In [1]:
import pandas as pd
import numpy as np
import joblib
from xgboost import XGBClassifier

In [2]:
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if root_path not in sys.path:
    sys.path.append(root_path)
from fileDir import getDataDir, getModelDir, getPredDir

In [3]:
TEST_PATH = getDataDir("test")
XGB_MODEL_PATH = getModelDir("model_xgboost", 3, True)
XGB_SCALER_PATH = getModelDir("scaler_model_xgboost", 3, True)
XGB_FEATURES_PATH = getModelDir("train_features_model_xgboost", 3, True)

XGB_PRED_PATH = getPredDir(3, "prediction_xgb")

Load Models and Test

In [4]:
xgb_model: XGBClassifier = joblib.load(XGB_MODEL_PATH)
xgb_scaler = joblib.load(XGB_SCALER_PATH)
xgb_train_features = joblib.load(XGB_FEATURES_PATH)

df = pd.read_csv(TEST_PATH)

ids = df["ID"]

FileNotFoundError: [Errno 2] No such file or directory: '../../model/v3/model_xgboost.pkl'

Prepare Test

In [ ]:
df["pms_i_ymd"] = pd.to_datetime(df["pms_i_ymd"], errors="coerce")
df["date_of_birth"] = pd.to_datetime(df["date_of_birth"], errors="coerce")
df["age"] = ((df["pms_i_ymd"] - df["date_of_birth"]).dt.days / 365).astype(float)
df["years_at_residence"] = df["living_period_year"] + df["living_period_month"] / 12
df["years_at_job"] = df["c_number_of_working_year"] + df["c_number_of_working_month"] / 12
df["total_income"] = df["c_monthly_salary"] + df["r_additional_income"] + df["r_spouse_income"]
df["debt_to_income_ratio"] = df["r_allloan_amount"] / (df["total_income"] + 1)
df["loan_burden_per_income"] = df["r_expected_credit_limit"] / (df["total_income"] + 1)

df_original = df.copy()
columns_to_remove = ['Area', 'Province', 'Shop Name','marital_status',
       'number_of_children', 'postal_code', 'number_of_resident', 'c_postal_code',
       'c_number_of_employee','media', 'place_for_sending_information', 'r_generalcode4',
       'r_generalcode5' ]
removed_data = df[columns_to_remove].copy()
df = df.drop(columns=columns_to_remove)

df = df.dropna()

columns_to_remove2 = ["pms_i_ymd", "date_of_birth", "living_period_year",
         "living_period_month", "c_number_of_working_year",
         "c_number_of_working_month"]
removed_data2 = df[columns_to_remove2].copy()
df = df.drop(columns=columns_to_remove2)

xgb_train_df = df

In [ ]:
# Allign Features
xgb_train_df = pd.get_dummies(xgb_train_df, drop_first=True)

for col in xgb_train_features:
    if col not in xgb_train_df.columns:
        xgb_train_df[col] = 0

xgb_train_df = xgb_train_df[xgb_train_features]


# Scaler
X_test_xgb = xgb_scaler.transform(xgb_train_df)

Predict

In [ ]:
xgb_y_proba = xgb_model.predict_proba(X_test_xgb)[:, 1]

xgb_threshold = 0.137
xgb_pred = (xgb_y_proba >= xgb_threshold).astype(int)

Export Output

In [ ]:
xgb_output_df = pd.DataFrame({
    "ID": ids,
    "default_12month": xgb_pred
})

xgb_output_df.to_csv(XGB_PRED_PATH, index=False)